In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
# import sklearn tools for commodity

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Binary Classification

## Logistic Regression

### Diabetes dataset

In [ ]:
# Read the dataset

dataset = pd.read_csv('data/diabetes_database.csv')

In [ ]:
# Extract features and labels as numpy arrays for convenience

X = dataset.drop("Outcome", axis=1).to_numpy()
y = dataset.Outcome.to_numpy()

# rescale the features
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
# Choose train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
random_state=42)

X_train.shape, X_test.shape

In [ ]:
# Create the model; allow a maximum of 1000 iterations to the algorithm to find the best solution

log_reg = LogisticRegression(max_iter=1000)

In [ ]:
# train the model

log_reg.fit(X_train, y_train)

In [ ]:
# get the predictions 
y_test_predict = log_reg.predict(X_test)

# print the accuracy score, or fraction of fraction of correctly classified samples
print(f"Accuracy on Test Set: {accuracy_score(y_test, y_test_predict) * 100:.2f}%")

Any accuracy score between $70\%$ and $90\%$ is a good performace test

In [ ]:
# Let us plot the confusion matrix

CM = confusion_matrix(y_test, y_test_predict, normalize="true", labels=log_reg.classes_)
disp = ConfusionMatrixDisplay(CM, display_labels=["Healthy", "Diabetic"])


`log_reg.classes_` stores the classes observed by the algorithm

In [ ]:
# make a plot

fig, ax = plt.subplots(figsize=(4, 4), facecolor="white", constrained_layout=True)
disp.plot(ax=ax)
plt.show()

### Star dataset

In [ ]:
# read dataset
stars = pd.read_csv("data/Stars.csv", delimiter=";")

# ordinal encoder for one variable (labels)

ordinal = LabelEncoder()
ordinal.fit(stars.Type)

y = ordinal.transform(stars.Type)

# making a dictionary of types to numbers

types = list(set(stars.Type))
numbers = ordinal.transform(types)

class_names = []

for i in np.argsort(numbers):
    class_names.append(types[i])

# OneHot encoder for the features "Color" and "Spectral_Class"

ohc = OneHotEncoder()

ohc.fit(stars[["Color", "Spectral_Class"]])

encoded_features = ohc.transform(stars[["Color", "Spectral_Class"]])

X = stars.drop(["Type", "Color", "Spectral_Class"], axis=1).to_numpy()

X = np.hstack([X, encoded_features.toarray()])

In [ ]:
# define the scaler
scaler = StandardScaler()
# train the scaler
scaler.fit(X)

X = scaler.transform(X)

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# PCA dimensional reduction

C = 10
pca = PCA(n_components=C)
pca.fit(X)

print(f'Total number of components used after PCA : {pca.n_components_}')

pca.explained_variance_ratio_.sum()

In [ ]:
# Transform the train and test sets

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

#### Logistic regression

In [ ]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_pca, y_train)

In [ ]:
y_test_predict = log_reg.predict(X_test_pca)

# print the accuracy score, or fraction of fraction of correctly classified samples
print(f"Accuracy on Test Set: {accuracy_score(y_test, y_test_predict) * 100:.2f}%")

Perfect accuracy!

In [ ]:
# Confusion matrix

CM = confusion_matrix(y_test, y_test_predict, normalize="true")
disp = ConfusionMatrixDisplay(CM, display_labels=class_names)

fig, ax = plt.subplots(figsize=(8, 8), facecolor="white", constrained_layout=True)
disp.plot(ax=ax)
plt.show()

### Higgs Boson dataset

In [ ]:
bosons = pd.read_csv("data/higgs.csv", delimiter=";")

In [ ]:
bosons_train, bosons_test = train_test_split(bosons, test_size=0.2, random_state=423)

X_train = bosons_train.drop("Event", axis=1).to_numpy()
y_train = bosons_train["Event"].copy().to_numpy()

X_test = bosons_test.drop("Event", axis=1).to_numpy()
y_test = bosons_test["Event"].copy().to_numpy()

X_train.shape, y_train.shape

In [ ]:
# Create and train the model

log_reg = LogisticRegression(max_iter=1000)

log_reg.fit(X_train, y_train)

In [ ]:
# get the predictions 
y_test_predict = log_reg.predict(X_test)

# print the accuracy score, or fraction of fraction of correctly classified samples
print(f"Accuracy on Test Set: {accuracy_score(y_test, y_test_predict) * 100:.2f}%")

Accuracy not is not very good

In [ ]:
# Let us plot the confusion matrix

CM = confusion_matrix(y_test, y_test_predict, normalize="true", labels=log_reg.classes_)
disp = ConfusionMatrixDisplay(CM)

In [ ]:
# make a plot

fig, ax = plt.subplots(figsize=(4, 4), facecolor="white", constrained_layout=True)
disp.plot(ax=ax)
plt.show()

It is good at predicting positive events, but it also provides a lot of false positives

Let us try a scaling

In [ ]:
# define the scaler
scaler = StandardScaler()
# train the scaler
scaler.fit(X_train)

In [ ]:
X_train_res = scaler.transform(X_train)
X_test_res = scaler.transform(X_test)

In [ ]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_res, y_train)

In [ ]:
y_test_predict = log_reg.predict(X_test_res)

# print the accuracy score, or fraction of fraction of correctly classified samples
print(f"Accuracy on Test Set: {accuracy_score(y_test, y_test_predict) * 100:.2f}%")

## $k$-Nearest Nighbors

### kNN Classifier

Let us try a KNN classifier with the same Higgs Boson data, to see if we can improve it

In [ ]:
# Let us try a KNN classifier, with K neighbors

K = 5
knn = KNeighborsClassifier(n_neighbors=K)

In [ ]:
# Train the model
knn.fit(X_train, y_train)

In [ ]:
# get the predictions
y_test_predict = knn.predict(X_test)

print(f"Accuracy on Test Set: {accuracy_score(y_test, y_test_predict) * 100:.2f}%")

Quite bad. Let us try changing the number of neighbors

In [ ]:
Ks = np.arange(2, 20).astype(int)

accuracies = []

for K in Ks:
    knn = KNeighborsClassifier(n_neighbors=K)
    knn.fit(X_train, y_train)
    y_test_predict = knn.predict(X_test)

    accuracies.append(accuracy_score(y_test, y_test_predict) * 100)


In [ ]:
plt.plot(Ks, accuracies, "o-", markersize=7)

plt.xlabel(r"$K$")
plt.ylabel("Accuracy")

No number of neighbors seem to provide a good accuracy. Some other tools should be used

### kNN Regressor

We can use kNN as a regressor, to predict a real value associated to datapoint characterized by some comples feature vector. 

As an example, consider the prediction of the critical temperature of superconductors.

In [ ]:
# read and transform the data

superconductors = pd.read_csv("data/superconductor.csv", delimiter=",")

X = superconductors.drop("critical_temp", axis=1).to_numpy()
y = superconductors["critical_temp"].copy().to_numpy()

# define the scaler
scaler = StandardScaler()

# train the scaler
scaler.fit(X)

# Scale the data

X = scaler.transform(X)

C = 30
pca = PCA(n_components=C)
pca.fit(X)

print(f'Total number of components used after PCA : {pca.n_components_}')

# Choose training and test sets

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.20, random_state=42)

# Transform the train and test sets

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

X_test_pca.shape, X_train_pca.shape


In [ ]:
# initiate a  knn regressor, looking a K=5 neighbors

K = 5
knn = KNeighborsRegressor(n_neighbors=K)

In [ ]:
# Train the model
knn.fit(X_train_pca, y_train)

In [ ]:
# Compute the prediction
y_test_predict = knn.predict(X_test_pca)

In [ ]:
# plot the prediction as a function of the real value

plt.scatter(y_test, y_test_predict, edgecolors='g')

plt.plot(y_test, y_test, "--r")

plt.xlabel("Real Temperature")
plt.ylabel("Predicted Temperature")

In [ ]:
# We can estimate the goodness of the fit using the mean absolute error of the prediction 
# with respect of the real value
mean_absolute_error(y_test, y_test_predict)

print(f"Mean Square Error: {mean_absolute_error(y_test, y_test_predict):.5}")

Not excellent, but not terribe either. As a first and dirty approximation, it could work

# Multiclass clasification

We can make classifications in more than two classes, using logistic regression or KNN

We use the Fashion MNIST dataset, in which datapoints can belong to 10 different classes

In [ ]:
train = pd.read_csv('data/fashion-mnist_train.csv')
test = pd.read_csv('data/fashion-mnist_test.csv')

In [ ]:
# The names corresponding to each label

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                   "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# Extract the sets as numpy arrays
X_train = train.drop("label", axis=1).to_numpy()
y_train = train.label.to_numpy()

X_test = test.drop("label", axis=1).to_numpy()
y_test = test.label.to_numpy()

# Normalize to [0,1]

X_train = X_train/X_train.max()
X_test = X_test/X_train.max()

In [ ]:
# Use only 125 components

C = 125
pca = PCA(n_components=C)
pca.fit(X_train)

print(f'Total number of components used after PCA : {pca.n_components_}')

pca.explained_variance_ratio_.sum()

# Transform the train and test sets

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

## KNN Classifier

In [ ]:
# Let us try a KNN classifier, with K neighbors
# Try to choose the best value of K

Ks = np.arange(2, 20).astype(int)

accuracies = []

for K in Ks:
    knn = KNeighborsClassifier(n_neighbors=K)
    knn.fit(X_train_pca, y_train)
    y_test_predict = knn.predict(X_test_pca)

    acc = accuracy_score(y_test, y_test_predict) * 100

    print("Accuracy ", K, ":", acc)

    accuracies.append(acc)


In [ ]:
plt.plot(Ks, accuracies, "o-", markersize=7)

plt.xlabel(r"$K$")
plt.ylabel("Accuracy")

Not very good accuracy

### Logistic regression

Logistic regression can be extended to multiclass classification

In [ ]:
# Create the model and train it

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_pca, y_train)

In [ ]:
y_test_predict = log_reg.predict(X_test_pca)

print(f"Accuracy on Test Set: {accuracy_score(y_test, y_test_predict) * 100:.2f}%")

Accuracy is OK

In [ ]:
# Compute the confusion matrix

CM = confusion_matrix(y_test, y_test_predict, normalize="true")
disp = ConfusionMatrixDisplay(CM, display_labels=class_names)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), facecolor="white", constrained_layout=True)
disp.plot(ax=ax)
plt.show()

Some classes ("Bag", "Coat" or "Ankle Boot") seem easy to classify. Others, such as "Shirt or "Pullover" get easily confused with "Coat".

# Dimensional reduction for visualization

We can use `tSNE` to perform a dimensional reduction of the data to $d=2$ and then a plot. We expect that objects within the same class should be close in space

In [ ]:
# Use tSNE for visualization Workstation
# Takes a lot of time, so we use only the first 10000 points of the train dataset

# Use the whole dataset PCA converted
X_tot_pca = X_train_pca[:10000]

In [ ]:
# Define and train the tSNE algorithm for a dimensional reduction to 2 dimensions
X_tsne = TSNE(n_components=2, random_state=12).fit_transform( X_tot_pca ) 

In [ ]:
# Plot the points of the dimensional reduction as a scatter plot

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(9, 7))

# every point has a color given by its label (the real value of the digit)
colors = y_train[:10000].astype(float)/y_train[:10000].astype(float).max()

c_ticks = np.sort(np.array(list(set(colors)), dtype=float))

plt.scatter(X_tsne[:,0], X_tsne[:,1], s=5, c=colors, cmap='jet')

cb = plt.colorbar(ticks=c_ticks)

_ = cb.ax.set_yticklabels(class_names)


Some misclassified points, but a very good result in average